In [5]:
%load_ext autoreload
%autoreload 2
import constrained as cd
from constrained.constraints import inset, aligned_vertically, distributed_vertically, aligned_horizontally, distributed_horizontally

In [2]:
import constrained.z3solver as z3solver
import constrained.ast as ast
import importlib
importlib.reload(z3solver)
importlib.reload(ast)
from functools import wraps

def element(func):
    @wraps(func)
    def element_wrapper(*args, **kwargs):
        original_defaults = func.__defaults__
        if func.__defaults__ is not None:
            defaults = []
            for default in func.__defaults__:
                if isinstance(default, ast.Placeholder):
                    defaults.append(ast.Variable())
                else:
                    defaults.append(default)
            func.__defaults__ = tuple(defaults)
        #args = [arg + 1 if isinstance(arg, Real) else arg for arg in args]
        #kwargs = dict([(name, val + 1) if isinstance(val, Real) else (name, val) for (name, val) in kwargs.items()])
        return_value = func(*args, **kwargs)
        func.__defaults__ = original_defaults
        return return_value
    return element_wrapper

@element
def foo(a=ast.Var, **kwargs):
    print(a)
    x = kwargs.get("x", ast.Var).var()
    print(x == 1)
    print(x == 2)

foo(ast.Variable())

ImportError: cannot import name 'Variable' from 'constrained.ast' (g:\CS\Private_Projects\constrained\constrained\ast.py)

In [17]:
from constrained.constraints import aligned_vertically, aligned_horizontally
from constrained.primitive import Rect, Circle, Group
from constrained.z3solver import to_z3

rect = Rect(width=5, height=5)
circle = Circle()
group = Group([rect, circle], [
    aligned_vertically([rect, circle]),
    aligned_horizontally([rect, circle]),
    rect.bounds.width == 2 * circle.radius
])
print([to_z3(c) for c in group.constraints])

[$166 + 5/2 == $171, $168 - $170 + (2*$170)/2 == $171, $167 + 5/2 == $172, $169 - $170 + (2*$170)/2 == $172, 5 == 2*$170, True, True, 0 < $170, [$173 <= $166, $173 <= $168 - $170, Or($173 == $166, $173 == $168 - $170)], [$174 <= $167, $174 <= $169 - $170, Or($174 == $167, $174 == $169 - $170)], [$173 + $175 >= $166 + 5, $173 + $175 >= $168 - $170 + 2*$170, Or($173 + $175 == $166 + 5,
   $173 + $175 == $168 - $170 + 2*$170)], [$174 + $176 >= $167 + 5, $174 + $176 >= $169 - $170 + 2*$170, Or($174 + $176 == $167 + 5,
   $174 + $176 == $169 - $170 + 2*$170)]]
